# Simulation of Electricity Consumption per Postal Code

#### Notebook Purpose:
The purpose of this notebook is to simulate the electricity consumption of postal codes by sampling from a conditional probability distribution fitted by a gaussian kernel density estimation. The samples represent the hourly electricity consumption of that particular household size. Using Zensus data, the number of different household sizes are aggregated for each postal code and sampled accordingly. The aggregation results in the hourly electricity consumption.

#### Dataset:
The analysis in this notebook is performed using multiple data sources. These sources provide the necessary information for simulating the electricity consumption of the households.
The datasets used in this analysis consists of the following:

1. Zensus Household Data: Demographic data from the Zensus 2011 collecting various detailed information about buildings, apartments and households on a 100m grid granularity. For this analysis the sizes of the households are relevant.
2. Open Power System Data: Electricity consumption data set from the OPen Powert System Data platform prividing hourly resoulition time series data on household load and solar generation. This data set spans two years worth of consumption data and we look at the residential households in Konstanz, specifically six residential electricity consumption profiles.

Using the Open Power System Data to fit the gaussian KDE and then sampling based on the household data lets us simulate the consumption of a postal code area.

### Load Data from Bucket

In [ ]:
# authenticate user

from google.colab import auth

auth.authenticate_user()

In [ ]:
# set parameters to use BigQuery Functionality

PROJECT_ID = 'solarinsight-383513' #@param {type: "string"}
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [ ]:
import google.cloud.bigquery as bq

client = bq.Client(project=PROJECT_ID)

In [ ]:
# used to convert the coordinate reference systems of the 100m grids

!pip install pyproj

from pyproj import Transformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 27.6 MB/s eta 0:00:00


In [ ]:
# copy the file from the bucket/folder_name/file to our colab notebook

!gsutil cp gs://bucket-quickstart-solarinsight/demographic_data/Haushalte100m.csv .

Copying gs://bucket-quickstart-solarinsight/demographic_data/Haushalte100m.csv...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Caught CTRL-C (signal 2) - exiting
^C


## Preparation of Household distribution per Postal Code

In [ ]:
# due to special characters we have to read the file with a certain encoding

import pandas as pd

df_hh = pd.read_csv('Haushalte100m.csv', encoding = "ISO-8859-1")

In [ ]:
df_hh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18806026 entries, 0 to 18806025
Data columns (total 7 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   Gitter_ID_100m      object
 1   Gitter_ID_100m_neu  object
 2   Merkmal             object
 3   Auspraegung_Code    int64 
 4   Auspraegung_Text    object
 5   Anzahl              int64 
 6   Anzahl_q            int64 
dtypes: int64(3), object(4)
memory usage: 1004.4+ MB


### Preprocessing and Filtering

In [ ]:
# we filter to only look at the characteristic household sizes and retrieve our relevant columns

df_hh = df_hh[df_hh['Merkmal'] == 'HHGROESS_KLASS']
df_hh = df_hh[['Gitter_ID_100m_neu', 'Auspraegung_Code', 'Anzahl']]
df_hh.columns = ['grid_id', 'hh_size', 'number']

In [ ]:
df_hh.head()

,grid_id,hh_size,number
4,CRS3035RES100mN2691700E4341100,1,3
5,CRS3035RES100mN2691700E4341100,2,3
11,CRS3035RES100mN2692300E4341100,1,4
15,CRS3035RES100mN2692400E4341200,2,3
19,CRS3035RES100mN2692600E4341000,2,3


### Transformation of Coordinate Reference Systems to map the 100m grids to Latitude and Longitude

In [ ]:
# from the grid ID we extract the relevant data to transform

df_hh['N'] = df_hh['grid_id'].str.extract(r'N(\d+)E')
df_hh['E'] = df_hh['grid_id'].str.extract(r'E(\d+)$')

transformer = Transformer.from_crs("EPSG:3035", "EPSG:4326")
df_hh['lat_long'] = df_hh.apply(lambda x: transformer.transform(x['N'], x['E']), axis = 1)
df_hh.head()

,grid_id,hh_size,number,N,E,lat_long
4,CRS3035RES100mN2691700E4341100,1,3,2691700,4341100,"(47.33857212533428, 10.265767901359636)"
5,CRS3035RES100mN2691700E4341100,2,3,2691700,4341100,"(47.33857212533428, 10.265767901359636)"
11,CRS3035RES100mN2692300E4341100,1,4,2692300,4341100,"(47.343972846106226, 10.265795488394295)"
15,CRS3035RES100mN2692400E4341200,2,3,2692400,4341200,"(47.34486972698704, 10.26712246536199)"
19,CRS3035RES100mN2692600E4341000,2,3,2692600,4341000,"(47.346676422756296, 10.264486860341993)"


In [ ]:
# we separate the latitude and longitude into two columns and save the relevant columns

df_hh['lat_long'] = df_hh['lat_long'].astype(str)
df_hh[['lat', 'long']] = df_hh['lat_long'].str.strip('()').str.split(',', expand=True)
df_hh['lat'] = df_hh['lat'].astype(float)
df_hh['long'] = df_hh['long'].astype(float)
df_hh = df_hh.drop(['N', 'E', 'lat_long'], axis=1)
df_hh.head()

,grid_id,hh_size,number,lat,long
4,CRS3035RES100mN2691700E4341100,1,3,47.338572,10.265768
5,CRS3035RES100mN2691700E4341100,2,3,47.338572,10.265768
11,CRS3035RES100mN2692300E4341100,1,4,47.343973,10.265795
15,CRS3035RES100mN2692400E4341200,2,3,47.344870,10.267122
19,CRS3035RES100mN2692600E4341000,2,3,47.346676,10.264487


### Insert household data into BigQuery Table

In [ ]:
# define the table name

table_name = PROJECT_ID + '.geo_data.hh_size_grid'
print('Creating table ' + table_name)

# Create the table
# - we use the same field names as in the original data set
table = bq.Table(table_name)
table.schema = (
        bq.SchemaField('grid_id',      'STRING'),
        bq.SchemaField('hh_size',      'INTEGER'),
        bq.SchemaField('number',      'INTEGER'),
        bq.SchemaField('lat',  'FLOAT' ),
        bq.SchemaField('long', 'FLOAT' )
)
client.create_table(table, exists_ok=True)

In [ ]:
print('Loading data into ' + table_name)
load_job = client.load_table_from_dataframe(df_hh, table)

if load_job.errors == None:
  print('Load complete!')
else:
  print(load_job.errors)

Loading data into solarinsight-383513.geo_data.hh_size_grid
Load complete!


### Mapping the 100m grid geometric points to Postal Code Polygons

In [ ]:
# the spatial temporal functions are used to see which postal code contains the geometric point of the respective 100m grid and this information is then saved as a table

%%bigquery --project solarinsight-383513

CREATE OR REPLACE TABLE `solarinsight-383513.geo_data.hh_size_plz5` AS
SELECT
  A.*,
  B.plz5
FROM
  `geo_data.hh_size_grid` AS A
JOIN
  `geo_data.plz5_polygone` AS B
ON
  ST_Contains(B.polygon, ST_GeogPoint(long, lat))

Query is running:   0%|          |

""


### Pivot number of x-person-sized households per postal code

In [ ]:
%%bigquery hh_plz --project solarinsight-383513
SELECT
hh_size, number, plz5
FROM
  `solarinsight-383513.geo_data.hh_size_plz5`


Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
# calculate the number of each household size in each postal code

number_of_people_per_household_per_plz = hh_plz.groupby(by = ['plz5', 'hh_size']).sum().reset_index()
number_of_people_per_household_per_plz

,plz5,hh_size,number
0,1067,1,4254
1,1067,2,2209
2,1067,3,555
3,1067,4,198
4,1067,5,30
...,...,...,...
47504,99998,2,690
47505,99998,3,455
47506,99998,4,258
47507,99998,5,12


less than 1% of the households are household size 6 or more, therefore can be approximated

In [ ]:
hh_plz.groupby(by = ['hh_size']).sum().reset_index()

,hh_size,number,plz5
0,1,13619460,75646055710
1,2,12122322,89479968687
2,3,5000879,59362339937
3,4,3388803,47841988573
4,5,692153,11698320844
5,6,213057,3464591292


In [ ]:
# pivot so that each row corresponds to one postal code with each column representing the number of the household sizes and how many of these are in the postal code

pivoted_hh_plz5 = number_of_people_per_household_per_plz.pivot(index='plz5', columns='hh_size', values='number').reset_index().fillna(0)
pivoted_hh_plz5 = pivoted_hh_plz5.rename_axis(None, axis=1)
pivoted_hh_plz5.columns = ['plz5', 'size_1', 'size_2', 'size_3', 'size_4', 'size_5', 'size_6_and_more']
pivoted_hh_plz5['plz5'] = pivoted_hh_plz5['plz5'].astype(str)
pivoted_hh_plz5

,plz5,size_1,size_2,size_3,size_4,size_5,size_6_and_more
0,1067,4254,2209,555,198,30,3
1,1069,9017,4741,1293,413,60,19
2,1097,4697,2652,1090,454,94,15
3,1099,7706,4104,1840,944,204,30
4,1108,579,965,453,302,33,6
...,...,...,...,...,...,...,...
8155,99988,334,574,396,292,83,15
8156,99991,484,679,459,261,18,27
8157,99994,800,1003,586,227,31,6
8158,99996,332,318,151,74,6,0


In [ ]:
# create BigQueryTable

table_name = PROJECT_ID + '.geo_data.hh_size_plz5_pivot'
print('Creating table ' + table_name)


table = bq.Table(table_name)
table.schema = (
        bq.SchemaField('plz5',      'STRING'),
        bq.SchemaField('size_1',      'INT64'),
        bq.SchemaField('size_2',      'INT64'),
        bq.SchemaField('size_3',  'INT64' ),
        bq.SchemaField('size_4', 'INT64' ),
        bq.SchemaField('size_5', 'INT64' ),
        bq.SchemaField('size_6_and_more', 'INT64' )
)
client.create_table(table, exists_ok=True)


Creating table solarinsight-383513.geo_data.hh_size_plz5_pivot


Table(TableReference(DatasetReference('solarinsight-383513', 'geo_data'), 'hh_size_plz5_pivot'))

In [ ]:
print('Loading data into ' + table_name)
load_job = client.load_table_from_dataframe(pivoted_hh_plz5, table)

if load_job.errors == None:
  print('Load complete!')
else:
  print(load_job.errors)

Loading data into solarinsight-383513.geo_data.hh_size_plz5_pivot
Load complete!


## Electricity Consumption Profile Preparation

### Load Data from Bucket

In [ ]:
!gsutil cp gs://bucket-quickstart-solarinsight/electricity_data/hourly_electricity_consumption.csv .

Copying gs://bucket-quickstart-solarinsight/electricity_data/hourly_electricity_consumption.csv...
\
Operation completed over 1 objects/14.9 MiB.                                     


### Preprocessing 6 residential Electricity Consumption Profiles

In [ ]:
# select the relevant columns of the profiles
# prepare month and hour to get the conditional probability distribution fit

import pandas as pd

consumption = pd.read_csv('hourly_electricity_consumption.csv')
consumption = consumption[['utc_timestamp', 'DE_KN_residential1_grid_import', 'DE_KN_residential1_pv',
'DE_KN_residential2_grid_import',
'DE_KN_residential3_grid_export', 'DE_KN_residential3_grid_import', 'DE_KN_residential3_pv',
'DE_KN_residential4_grid_export', 'DE_KN_residential4_grid_import', 'DE_KN_residential4_pv',
'DE_KN_residential5_grid_import',
'DE_KN_residential6_grid_export', 'DE_KN_residential6_grid_import', 'DE_KN_residential6_pv',]]
consumption.columns = ['timestamp', 'grid_import1', 'pv_generation1', 'grid_import2', 'grid_export3', 'grid_import3', 'pv_generation3', 'grid_export4', 'grid_import4', 'pv_generation4', 'grid_import5', 'grid_export6', 'grid_import6', 'pv_generation6']
consumption[consumption.columns.difference(['timestamp'])] = consumption[consumption.columns.difference(['timestamp'])].apply(lambda x: x.diff())
consumption['timestamp'] = pd.to_datetime(consumption['timestamp'])
consumption['month'] = consumption['timestamp'].dt.month
consumption['hour'] = consumption['timestamp'].dt.hour

profile_1 = consumption[['timestamp','month', 'hour', 'grid_import1', 'pv_generation1']].copy()
profile_2 = consumption[['timestamp','month', 'hour', 'grid_import2']].copy()
profile_3 = consumption[['timestamp','month', 'hour', 'grid_export3', 'grid_import3', 'pv_generation3']].copy()
profile_4 = consumption[['timestamp','month', 'hour', 'grid_export4', 'grid_import4', 'pv_generation4']].copy()
profile_5 = consumption[['timestamp','month', 'hour', 'grid_import5']].copy()
profile_6 = consumption[['timestamp','month', 'hour', 'grid_export6', 'grid_import6', 'pv_generation6']].copy()

In [ ]:
consumption[consumption['timestamp'].dt.year > 2016].head()

,timestamp,grid_import1,pv_generation1,grid_import2,grid_export3,grid_import3,pv_generation3,grid_export4,grid_import4,pv_generation4,grid_import5,grid_export6,grid_import6,pv_generation6,month,hour
18031,2017-01-01 00:00:00+00:00,0.361,0.0,0.155,0.0,0.325,0.0,0.0,1.371,0.0,0.780,0.0,0.310,0.0,1,0
18032,2017-01-01 01:00:00+00:00,0.350,0.0,0.130,0.0,0.245,0.0,0.0,0.798,0.0,0.210,0.0,0.220,0.0,1,1
18033,2017-01-01 02:00:00+00:00,0.369,0.0,0.170,0.0,0.250,0.0,0.0,1.224,0.0,0.190,0.0,0.485,0.0,1,2
18034,2017-01-01 03:00:00+00:00,0.415,0.0,0.230,0.0,0.265,0.0,0.0,0.749,0.0,0.180,0.0,0.430,0.0,1,3
18035,2017-01-01 04:00:00+00:00,0.441,0.0,0.160,0.0,0.265,0.0,0.0,1.231,0.0,0.182,0.0,0.365,0.0,1,4


### Profile 1

In [ ]:
# estimate the consumption depending on pv generation and grid export if necessary
# calculate some statistics to determine the profiles suitability

profile_1 = profile_1[~profile_1['grid_import1'].isna()].copy()
profile_1 = profile_1.fillna(0)
profile_1['consumption'] = profile_1['grid_import1']
print(profile_1.describe())

first_value = profile_1['timestamp'].iloc[0]
last_value = profile_1['timestamp'].iloc[-1]
days_difference = (last_value - first_value).days
print("\nNumber of days:" , days_difference)
print("Average yearly consumption:" , profile_1['consumption'].sum()/days_difference*365)

profile_1 = profile_1[['month', 'hour', 'consumption']]

              month          hour  grid_import1  pv_generation1   consumption
count  15872.000000  15872.000000  15872.000000    15872.000000  15872.000000
mean       6.782762     11.504032      0.573102        1.040935      0.573102
std        3.516874      6.923180      0.367954        1.889026      0.367954
min        1.000000      0.000000      0.029000        0.000000      0.029000
25%        4.000000      6.000000      0.375000        0.000000      0.375000
50%        7.000000     12.000000      0.447000        0.000000      0.447000
75%       10.000000     18.000000      0.614000        1.115000      0.614000
max       12.000000     23.000000      4.083000        8.197000      4.083000

Number of days: 661
Average yearly consumption: 5022.9058093797275


### Profile 2

In [ ]:
# estimate the consumption depending on pv generation and grid export if necessary
# calculate some statistics to determine the profiles suitability

profile_2 = profile_2[~profile_2['grid_import2'].isna()].copy()
profile_2 = profile_2.fillna(0)
profile_2['consumption'] = profile_2['grid_import2'].copy()
print(profile_2.describe())

first_value = profile_2['timestamp'].iloc[0]
last_value = profile_2['timestamp'].iloc[-1]
days_difference = (last_value - first_value).days
print("\nNumber of days:" , days_difference)
print("Average yearly consumption:" , profile_2['consumption'].sum()/days_difference*365)

profile_2 = profile_2[['month', 'hour', 'consumption']]

              month          hour  grid_import2   consumption
count  15797.000000  15797.000000  15797.000000  15797.000000
mean       6.928784     11.500158      0.284572      0.284572
std        3.387718      6.921361      0.286021      0.286021
min        1.000000      0.000000      0.040000      0.040000
25%        4.000000      6.000000      0.137000      0.137000
50%        7.000000     12.000000      0.185000      0.185000
75%       10.000000     17.000000      0.300000      0.300000
max       12.000000     23.000000      4.987000      4.987000

Number of days: 658
Average yearly consumption: 2493.6367325227966


### Profile 3

In [ ]:
# estimate the consumption depending on pv generation and grid export if necessary
# calculate some statistics to determine the profiles suitability
# remove the extreme outliers and adjust

profile_3 = profile_3[~profile_3['grid_import3'].isna()].copy()
profile_3 = profile_3.fillna(0)
profile_3.loc[profile_3['grid_export3'] > profile_3['pv_generation3'], 'grid_export3'] = profile_3['pv_generation3']
profile_3 = profile_3[profile_3['pv_generation3'] < 15]
profile_3 = profile_3[profile_3['grid_import3'] < 15]
profile_3['consumption'] = profile_3['grid_import3'] + profile_3['pv_generation3'] - profile_3['grid_export3']
print(profile_3.describe())

first_value = profile_3['timestamp'].iloc[0]
last_value = profile_3['timestamp'].iloc[-1]
days_difference = (last_value - first_value).days
print("\nNumber of days:" , days_difference)
print("Average yearly consumption:" , profile_3['consumption'].sum()/days_difference*365)

profile_3 = profile_3[['month', 'hour', 'consumption']]

              month          hour  grid_export3  grid_import3  pv_generation3  \
count  11895.000000  11895.000000  11895.000000  11895.000000    11895.000000   
mean       6.022699     11.505507      0.393592      0.418012        0.576422   
std        3.143209      6.919833      0.728513      0.454383        0.906446   
min        1.000000      0.000000      0.000000      0.000000        0.000000   
25%        4.000000      6.000000      0.000000      0.064000        0.000000   
50%        6.000000     12.000000      0.000000      0.252000        0.012000   
75%        8.000000     18.000000      0.445500      0.629000        0.859000   
max       12.000000     23.000000      3.409000      3.060000        3.730000   

        consumption  
count  11895.000000  
mean       0.600842  
std        0.453370  
min        0.000000  
25%        0.252000  
50%        0.465000  
75%        0.820500  
max        3.548000  

Number of days: 495
Average yearly consumption: 5270.0218989899


### Profile 4

In [ ]:
# estimate the consumption depending on pv generation and grid export if necessary
# calculate some statistics to determine the profiles suitability

profile_4 = profile_4[~profile_4['grid_import4'].isna()].copy()
profile_4 = profile_4.fillna(0)
profile_4.loc[profile_4['grid_export4'] > profile_4['pv_generation4'], 'grid_export4'] = profile_4['pv_generation4']
profile_4['consumption'] = profile_4['grid_import4'] + profile_4['pv_generation4'] - profile_4['grid_export4']
print(profile_4.describe())

first_value = profile_4['timestamp'].iloc[0]
last_value = profile_4['timestamp'].iloc[-1]
days_difference = (last_value - first_value).days
print("\nNumber of days:" , days_difference)
print("Average yearly consumption:" , profile_4['consumption'].sum()/days_difference*365)

profile_4 = profile_4[['month', 'hour', 'consumption']]

              month          hour  grid_export4  grid_import4  pv_generation4  \
count  20359.000000  20359.000000  20359.000000  20359.000000    20359.000000   
mean       6.742866     11.502923      0.932324      0.503317        1.207154   
std        3.677788      6.923060      1.795187      0.552071        2.040357   
min        1.000000      0.000000      0.000000      0.000000        0.000000   
25%        3.000000      6.000000      0.000000      0.070000        0.000000   
50%        7.000000     12.000000      0.000000      0.294000        0.010000   
75%       10.000000     18.000000      0.874000      0.797000        1.544500   
max       12.000000     23.000000      8.371000      4.898000        8.785000   

        consumption  
count  20359.000000  
mean       0.778147  
std        0.731599  
min        0.000000  
25%        0.240000  
50%        0.612000  
75%        0.976000  
max        8.836000  

Number of days: 848
Average yearly consumption: 6818.908478773588


### Profile 5

In [ ]:
# estimate the consumption depending on pv generation and grid export if necessary
# calculate some statistics to determine the profiles suitability

profile_5 = profile_5[~profile_5['grid_import5'].isna()].copy()
profile_5 = profile_5.fillna(0)
profile_5['consumption'] = profile_5['grid_import5'].copy()
print(profile_5.describe())

first_value = profile_5['timestamp'].iloc[0]
last_value = profile_5['timestamp'].iloc[-1]
days_difference = (last_value - first_value).days
print("\nNumber of days:" , days_difference)
print("Average yearly consumption:" , profile_5['consumption'].sum()/days_difference*365)

profile_5 = profile_5[['month', 'hour', 'consumption']]

              month          hour  grid_import5   consumption
count  30803.000000  30803.000000  30803.000000  30803.000000
mean       6.396617     11.501964      0.284834      0.284834
std        3.608429      6.922101      0.228488      0.228488
min        1.000000      0.000000      0.029000      0.029000
25%        3.000000      6.000000      0.128000      0.128000
50%        6.000000     12.000000      0.210000      0.210000
75%       10.000000     18.000000      0.361000      0.361000
max       12.000000     23.000000      2.220000      2.220000

Number of days: 1283
Average yearly consumption: 2496.0409781761496


### Profile 6

In [ ]:
# estimate the consumption depending on pv generation and grid export if necessary
# calculate some statistics to determine the profiles suitability

profile_6 = profile_6[~profile_6['grid_import6'].isna()].copy()
profile_6 = profile_6.fillna(0)
profile_6.loc[profile_6['grid_export6'] > profile_6['pv_generation6'], 'grid_export6'] = profile_6['pv_generation6']
profile_6['consumption'] = profile_6['grid_import6'] + profile_6['pv_generation6'] - profile_6['grid_export6']
print(profile_6.describe())

first_value = profile_6['timestamp'].iloc[0]
last_value = profile_6['timestamp'].iloc[-1]
days_difference = (last_value - first_value).days
print("\nNumber of days:" , days_difference)
print("Average yearly consumption:" , profile_6['consumption'].sum()/days_difference*365)

profile_6 = profile_6[['month', 'hour', 'consumption']]

              month          hour  grid_export6  grid_import6  pv_generation6  \
count  21533.000000  21533.000000  21533.000000  21533.000000    21533.000000   
mean       6.411369     11.501974      0.157495      0.341193        0.951813   
std        3.696653      6.922789      0.550413      0.365684        1.778876   
min        1.000000      0.000000      0.000000      0.000000        0.000000   
25%        3.000000      6.000000      0.000000      0.020000        0.000000   
50%        6.000000     12.000000      0.000000      0.245000        0.000000   
75%       10.000000     18.000000      0.010000      0.590000        0.930000   
max       12.000000     23.000000      3.690000      2.801000        7.646000   

        consumption  
count  21533.000000  
mean       1.135512  
std        1.418454  
min        0.000000  
25%        0.260000  
50%        0.645000  
75%        1.245000  
max        9.211000  

Number of days: 897
Average yearly consumption: 9949.39542920848


### Mapping Electricity Consumption Profiles to Household sizes

We estimate households consume approximately this amount per year as per common mean values

https://www.check24.de/strom-gas/ratgeber/stromverbrauch/

-	1 person 1500 kwh/year
- 2 people 2500 kwh/year
- 3 people 3500 kwh/year
- 4 people 4250 kwh/year
- 5 people 5000 kwh/year
- 6 people 5750 kwh/year

More than 6 people are replaced and estimated as household sizes of 6 people with a constant growth, as this category makes up less than 1% of all household profiles and is unlikely to happen in a common family composition of two parents and several children


In [ ]:
# profiles have a certain average kwh/year consumption, transform to fit the different household sizes
# the comments show how much these profiles after transformation consume per year

profile_hh2 = pd.concat([profile_2, profile_5]) # 2495 kwh/year
profile_hh1 = profile_hh2.copy()
profile_hh1['consumption'] = profile_hh1['consumption']/1.7 # 1471 kwh/year
profile_4['consumption'] = profile_4['consumption']/2 # 3419 kwh/year
profile_3['consumption'] = profile_3['consumption']/1.4  # 4188 kwh/year
profile_6['consumption'] = profile_6['consumption']/1.75 # 5686 kwh/year

In [ ]:
hh_1 = profile_hh1.copy() # 1471 kwh/year
hh_2 = profile_hh2.copy() # 2495 kwh/year
hh_3 = profile_4.copy() # 3419 kwh/year
hh_4 = profile_3.copy() # 4188 kwh/year
hh_5 = profile_1.copy() # 5023 kwh/year
hh_6 = profile_6.copy() # 5686 kwh/year

## Sampling from Conditional Probability Distribution fit using Gaussian Kernel Density Estimation

### Load Household Data

In [ ]:
%%bigquery hh_plz --project solarinsight-383513
SELECT
plz5, size_1, size_2, size_3, size_4, size_5, size_6_and_more
FROM
  `solarinsight-383513.geo_data.hh_size_plz5_pivot`

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
hh_plz

,plz5,size_1,size_2,size_3,size_4,size_5,size_6_and_more
0,1776,84,123,30,6,0,0
1,1825,143,156,72,58,0,0
2,1847,374,531,257,120,0,0
3,2799,136,196,126,36,0,0
4,3052,87,172,104,42,0,0
...,...,...,...,...,...,...,...
8155,60431,4876,3340,1479,1100,425,251
8156,60439,6481,4999,2454,1754,639,252
8157,21107,4862,2671,1383,897,429,252
8158,60435,5912,3874,1801,1408,529,254


### Run Simulation

Function that returns a Gaussian KDE conditioned on month and hour

Rather small bandwidth of 0.1 allows to fit more closely to training data and prevent too long tails going into negative consumption

In [ ]:
from sklearn.neighbors import KernelDensity

def conditional_kde(df, month, hour):
  kde = KernelDensity(kernel='gaussian', bandwidth = 0.1).fit(df[(df['month'] == month) & (df['hour'] == hour)][['consumption']])
  return kde


In [ ]:
# run only in second iteration as we want to continue simulating from the latest already simulated date on
# for the first iteration just delete the old table and skip this cell

%%bigquery latest_date --project solarinsight-383513
SELECT
MAX(datetime)
FROM
  `solarinsight-383513.geo_data.postal_code_consumption`

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
latest_date # of the simulation stored in the table

,f0_
0,2024-07-01 23:00:00+00:00


In [ ]:
# for each of the remaining simulation days:
#   simulate the 24 hours of the day of each postal code and all the household sizes
#   estimate the electricity consumption for each of the household sizes by sampling from the conditional probability distribution fit
#   each household size has a different probability distribution fit

from datetime import datetime, timedelta
import pandas_gbq
import numpy as np

original_date = pd.to_datetime('2023-07-01') # start date of the simulation is set here
start_date = pd.Timestamp((max(original_date, pd.Timestamp(latest_date.iloc[0][0]).replace(tzinfo=None)) + timedelta(days=1)).date()) # run only from second iteration on, skip if table deleted
# start_date = original_date # run only in first iteration
end_date = original_date + pd.DateOffset(years=1) # 1 year of simulation time

for date in pd.date_range(start=start_date, end=end_date, freq='D'):

  date_df = pd.DataFrame(range(24), columns = ['hour'])
  date_df['month'] = pd.to_datetime(date).month
  date_df['datetime'] = date + pd.to_timedelta(date_df['hour'], unit='h')

  date_df = date_df.assign(dummy=1).merge(hh_plz.assign(dummy=1), on='dummy', how='outer').drop('dummy', axis=1)

  for hour in range(0, 24):

    # conditioned on each hour and the month estimate the KDE and sample and aggregate for all of the distinct household sizes

    kde = conditional_kde(hh_1, pd.to_datetime(date).month, hour)
    date_df.loc[(date_df['hour'] == hour), 'size_1_consumption'] = date_df.loc[(date_df['hour'] == hour), 'size_1'].apply(lambda x: np.sum(kde.sample(x)))

    kde = conditional_kde(hh_2, pd.to_datetime(date).month, hour)
    date_df.loc[(date_df['hour'] == hour), 'size_2_consumption'] = date_df.loc[(date_df['hour'] == hour), 'size_2'].apply(lambda x: np.sum(kde.sample(x)))

    kde = conditional_kde(hh_3, pd.to_datetime(date).month, hour)
    date_df.loc[(date_df['hour'] == hour), 'size_3_consumption'] = date_df.loc[(date_df['hour'] == hour), 'size_3'].apply(lambda x: np.sum(kde.sample(x)))

    kde = conditional_kde(hh_4, pd.to_datetime(date).month, hour)
    date_df.loc[(date_df['hour'] == hour), 'size_4_consumption'] = date_df.loc[(date_df['hour'] == hour), 'size_4'].apply(lambda x: np.sum(kde.sample(x)))

    kde = conditional_kde(hh_5, pd.to_datetime(date).month, hour)
    date_df.loc[(date_df['hour'] == hour), 'size_5_consumption'] = date_df.loc[(date_df['hour'] == hour), 'size_5'].apply(lambda x: np.sum(kde.sample(x)))

    kde = conditional_kde(hh_6, pd.to_datetime(date).month, hour)
    date_df.loc[(date_df['hour'] == hour), 'size_6_consumption'] = date_df.loc[(date_df['hour'] == hour), 'size_6_and_more'].apply(lambda x: np.sum(kde.sample(x)))

# aggregate and save the table in BigQuery for each simulated day

  date_df['total_consumption'] = date_df['size_1_consumption'] + date_df['size_2_consumption'] + date_df['size_3_consumption'] + date_df['size_4_consumption'] + date_df['size_5_consumption'] + date_df['size_6_consumption']
  date_df = date_df[['datetime', 'plz5', 'total_consumption']]

  table_name = 'geo_data.postal_code_consumption'

  pandas_gbq.to_gbq(date_df, table_name, project_id=PROJECT_ID, if_exists='append')



100%|██████████| 1/1 [00:00<00:00, 3177.50it/s]


# Possible future inclusion

This part explores the remaining Zensus data e.g. Apartment and Building data which can in the future be integrated to have a more refined estimation of the electricity consumption of the postal code areas

## Gebäude aggregation

In [ ]:
#!gsutil cp gs://bucket-quickstart-solarinsight/demographic_data/Geb100m.csv .
#!gsutil cp gs://bucket-quickstart-solarinsight/demographic_data/Wohnungen100m.csv .

In [ ]:
import pandas as pd

df_geb = pd.read_csv('Geb100m.csv', encoding = "ISO-8859-1")
# df_wohn = pd.read_csv('Wohnungen100m.csv', encoding = "ISO-8859-1")

In [ ]:
df_geb[df_geb['Merkmal'] == 'ZAHLWOHNGN_HHG'].info()
# dataset of Wohnung has 4.9 million observations

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3440249 entries, 2 to 24379874
Data columns (total 7 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   Gitter_ID_100m      object
 1   Gitter_ID_100m_neu  object
 2   Merkmal             object
 3   Auspraegung_Code    int64 
 4   Auspraegung_Text    object
 5   Anzahl              int64 
 6   Anzahl_q            int64 
dtypes: int64(3), object(4)
memory usage: 210.0+ MB


In [ ]:
df_geb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24379875 entries, 0 to 24379874
Data columns (total 7 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   Gitter_ID_100m      object
 1   Gitter_ID_100m_neu  object
 2   Merkmal             object
 3   Auspraegung_Code    int64 
 4   Auspraegung_Text    object
 5   Anzahl              int64 
 6   Anzahl_q            int64 
dtypes: int64(3), object(4)
memory usage: 1.3+ GB


In [ ]:
df_geb = df_geb[df_geb['Merkmal'] == ' ZAHLWOHNGN_HHG']
df_geb = df_geb[['Gitter_ID_100m_neu', 'Auspraegung_Code', 'Anzahl']]
df_geb.columns = ['grid_id', 'hh_size', 'number']

In [ ]:
df_hh.head()

In [ ]:
df_hh['N'] = df_hh['grid_id'].str.extract(r'N(\d+)E')
df_hh['E'] = df_hh['grid_id'].str.extract(r'E(\d+)$')

transformer = Transformer.from_crs("EPSG:3035", "EPSG:4326")
df_hh['lat_long'] = df_hh.apply(lambda x: transformer.transform(x['N'], x['E']), axis = 1)
df_hh.head()

In [ ]:
df_hh['lat_long'] = df_hh['lat_long'].astype(str)
df_hh[['lat', 'long']] = df_hh['lat_long'].str.strip('()').str.split(',', expand=True)
df_hh['lat'] = df_hh['lat'].astype(float)
df_hh['long'] = df_hh['long'].astype(float)
df_hh = df_hh.drop(['N', 'E', 'lat_long'], axis=1)
df_hh.head()

In [ ]:
df_hh.to_csv('hh_lat_long.csv',index=False)
!gsutil cp hh_lat_long.csv gs://bucket-quickstart-solarinsight/geo_data/

## Wohnung aggregation

In [ ]:
import pandas as pd

df_wohn = pd.read_csv('Wohnungen100m.csv', encoding = "ISO-8859-1")

In [ ]:
df_wohn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51342617 entries, 0 to 51342616
Data columns (total 7 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   Gitter_ID_100m      object
 1   Gitter_ID_100m_neu  object
 2   Merkmal             object
 3   Auspraegung_Code    int64 
 4   Auspraegung_Text    object
 5   Anzahl              int64 
 6   Anzahl_q            int64 
dtypes: int64(3), object(4)
memory usage: 2.7+ GB


In [ ]:
df_wohn = df_wohn[df_wohn['Merkmal'].isin(['ZAHLWOHNGN_HHG', 'BAUJAHR_MZ', 'RAUMANZAHL', 'WOHNFLAECHE_10S'])]
df_wohn = df_wohn[['Gitter_ID_100m_neu', 'Merkmal', 'Auspraegung_Text', 'Anzahl']]
df_wohn.columns = ['grid_id', 'feature', 'text', 'number']

In [ ]:
df_wohn.head()

,grid_id,feature,text,number
3,CRS3035RES100mN2686500E4335700,ZAHLWOHNGN_HHG,1 Wohnung,3
5,CRS3035RES100mN2689100E4337000,BAUJAHR_MZ,1949 - 1978,3
11,CRS3035RES100mN2691200E4341200,BAUJAHR_MZ,Vor 1919,3
16,CRS3035RES100mN2691200E4341200,ZAHLWOHNGN_HHG,2 Wohnungen,3
18,CRS3035RES100mN2691700E4341100,BAUJAHR_MZ,1949 - 1978,3


In [ ]:
df_wohn['N'] = df_wohn['grid_id'].str.extract(r'N(\d+)E')
df_wohn['E'] = df_wohn['grid_id'].str.extract(r'E(\d+)$')

transformer = Transformer.from_crs("EPSG:3035", "EPSG:4326")
df_wohn['lat_long'] = df_wohn.apply(lambda x: transformer.transform(x['N'], x['E']), axis = 1)
df_wohn.head()

,grid_id,feature,text,number,N,E,lat_long
3,CRS3035RES100mN2686500E4335700,ZAHLWOHNGN_HHG,1 Wohnung,3,2686500,4335700,"(47.29191605392987, 10.19419327973013)"
5,CRS3035RES100mN2689100E4337000,BAUJAHR_MZ,1949 - 1978,3,2689100,4337000,"(47.31528766587454, 10.211461739121242)"
11,CRS3035RES100mN2691200E4341200,BAUJAHR_MZ,Vor 1919,3,2691200,4341200,"(47.33406827626757, 10.267067021719932)"
16,CRS3035RES100mN2691200E4341200,ZAHLWOHNGN_HHG,2 Wohnungen,3,2691200,4341200,"(47.33406827626757, 10.267067021719932)"
18,CRS3035RES100mN2691700E4341100,BAUJAHR_MZ,1949 - 1978,3,2691700,4341100,"(47.33857212533428, 10.265767901359636)"


In [ ]:
# due to resource limit export temporary results

df_wohn.to_csv('temp_wohn_lat_long.csv',index=False)


In [ ]:
import pandas as pd

df_wohn = pd.read_csv('temp_wohn_lat_long.csv', encoding = "ISO-8859-1")

In [ ]:
df_wohn['lat_long'] = df_wohn['lat_long'].astype(str)
df_wohn[['lat', 'long']] = df_wohn['lat_long'].str.strip('()').str.split(',', expand=True)
df_wohn['lat'] = df_wohn['lat'].astype(float)
df_wohn['long'] = df_wohn['long'].astype(float)
df_wohn = df_wohn.drop(['N', 'E', 'lat_long'], axis=1)
df_wohn.head()

,grid_id,feature,text,number,lat,long
0,CRS3035RES100mN2686500E4335700,ZAHLWOHNGN_HHG,1 Wohnung,3,47.291916,10.194193
1,CRS3035RES100mN2689100E4337000,BAUJAHR_MZ,1949 - 1978,3,47.315288,10.211462
2,CRS3035RES100mN2691200E4341200,BAUJAHR_MZ,Vor 1919,3,47.334068,10.267067
3,CRS3035RES100mN2691200E4341200,ZAHLWOHNGN_HHG,2 Wohnungen,3,47.334068,10.267067
4,CRS3035RES100mN2691700E4341100,BAUJAHR_MZ,1949 - 1978,3,47.338572,10.265768


In [ ]:
df_wohn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21656180 entries, 0 to 21656179
Data columns (total 6 columns):
 #   Column   Dtype  
---  ------   -----  
 0   grid_id  object 
 1   feature  object 
 2   text     object 
 3   number   int64  
 4   lat      float64
 5   long     float64
dtypes: float64(2), int64(1), object(3)
memory usage: 991.3+ MB


In [ ]:
df_wohn.to_csv('wohn_lat_long.csv',index=False)
!gsutil cp wohn_lat_long.csv gs://bucket-quickstart-solarinsight/geo_data/

Copying file://wohn_lat_long.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

-
Operation completed over 1 objects/1.9 GiB.                                      


## Query number of x-sized houses

In [ ]:
%%bigquery dfl --project solarinsight-383513
SELECT
text, number, plz5
FROM
  `solarinsight-383513.geo_data.wohn_plz5`
WHERE
  feature = 'ZAHLWOHNGN_HHG'


Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
dfl.head()

,text,number,plz5
0,1 Wohnung,3,83052
1,1 Wohnung,3,14712
2,1 Wohnung,3,27476
3,1 Wohnung,3,15749
4,1 Wohnung,3,27619


In [ ]:
number_of_apartments_per_house_per_plz = dfl.groupby(by = ['plz5', 'text']).sum().reset_index()

In [ ]:
number_of_apartments_per_house_per_plz.head()

,plz5,text,number
0,1067,1 Wohnung,12
1,1067,13 und mehr Wohnungen,4507
2,1067,2 Wohnungen,28
3,1067,3 - 6 Wohnungen,125
4,1067,7 - 12 Wohnungen,3060


In [ ]:
import numpy as np

def apply_mapping(value):
    replacement_mapping = {
        '1 Wohnung': '1',
        '2 Wohnungen': '2',
        '3 - 6 Wohnungen': np.random.choice(['3', '4', '5', '6'], p=[0.3, 0.3, 0.2, 0.2]),
        '7 - 12 Wohnungen': np.random.choice(['7', '8', '9', '10', '11', '12'], p=[0.2, 0.2, 0.2, 0.2, 0.1, 0.1]),
        '13 und mehr Wohnungen': '13'
    }
    return replacement_mapping.get(value, value)

# Apply the mapping using the apply() method
number_of_apartments_per_house_per_plz['text'] = number_of_apartments_per_house_per_plz['text'].apply(apply_mapping)

In [ ]:
number_of_apartments_per_house_per_plz['text'].value_counts()

2     8156
1     8142
13    5415
3     2489
4     2391
5     1664
6     1603
9     1496
8     1489
10    1449
7     1385
12     716
11     680
Name: text, dtype: int64

In [ ]:
number_of_apartments_per_house_per_plz.pivot(index='plz5', columns='text', values='number').reset_index().fillna(0)

text,plz5,1,10,11,12,13,2,3,4,5,6,7,8,9
0,1067,12,0,0,0,4507,28,0,125,0,0,0,3060,0
1,1069,181,0,0,0,10361,87,0,757,0,0,0,0,4898
2,1097,104,0,0,0,4790,92,0,0,0,663,0,3947,0
3,1099,324,0,0,0,6015,326,1902,0,0,0,7147,0,0
4,1108,1343,0,0,0,57,776,0,435,0,0,0,112,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8156,99988,995,0,0,0,0,673,0,0,214,0,0,0,58
8157,99991,1314,0,0,0,13,554,277,0,0,0,0,187,0
8158,99994,1452,0,0,0,39,708,323,0,0,0,745,0,0
8159,99996,398,0,0,0,19,215,0,0,0,329,160,0,0
